# Lesson 4: Mastering Data Retrieval in DynamoDB: GetItem and BatchGetItem

### Introduction and Context Recap

Welcome back! Having previously set up and configured tables in DynamoDB using Python's Boto3 library, today we focus on retrieving data from these tables. Retrieving data is crucial for any database system as it allows us to access and utilize stored data effectively. In this lesson, we concentrate on two pivotal operations—GetItem and BatchGetItem. Both are essential for efficient data retrieval, whether you're pulling a single item or multiple items based on their keys. By the end of this session, you'll be well-equipped to handle data retrieval tasks in DynamoDB proficiently.

#### Reading Data with "GetItem"

Retrieving data from a DynamoDB table is straightforward when you have the full primary key of the item you're interested in. Let's say we need details for *The Big New Movie* released in 2016 from our Movies table, where year and title make up the primary key.

For a simple read, you can retrieve the item like this:

```python
# Simple read
result = table.get_item(Key={'year': 2016, 'title': 'The Big New Movie'})
if 'Item' in result:
    print("Movie found:", result['Item'])
else:
    print("Movie not found.")
```

If you only need certain attributes, you can specify them using the `ProjectionExpression` parameter:

```python
# Attributes projection
result = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'},
    ProjectionExpression='title, genre',
)
if 'Item' in result:
    print("Projected attributes of the movie:", result['Item'])
```

For a strongly consistent read, add the `ConsistentRead` parameter:

```python
# Strongly consistent read
result = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'},
    ConsistentRead=True
)
if 'Item' in result:
    print("Consistently read movie details:", result['Item'])
```

#### Retrieving Multiple Items with BatchGetItem

For fetching several items known by their primary keys, DynamoDB provides the **BatchGetItem** API. This operation is different from GetItem, as it retrieves more than one item at a time from a table. When using BatchGetItem, you'll need to specify the full primary key for each item—this means both the partition key and sort key if the table has a composite primary key. Here’s an example of how to retrieve multiple items using their full primary keys:

```python
response = dynamodb.batch_get_item(
    RequestItems={
        'Movies': {
            'Keys': [
                {'year': 2016, 'title': 'The Big New Movie'},  # Full primary key (partition key and sort key)
                {'year': 2017, 'title': 'The Bigger, Newer Movie'}  # Full primary key
            ],
            'ConsistentRead': True
        }
    }
)
```

In the above snippet, each dictionary in the Keys array contains a complete primary key that identifies a unique item in the Movies table. Specifying the full primary key in your BatchGetItem request ensures precise and efficient retrieval of the items you need.

While BatchGetItem is a powerful tool for retrieving multiple items at once, it comes with several limitations:

- **Maximum Items:** You can fetch up to 25 items in a single batch request.
- **Maximum Data Retrieval:** The total amount of data retrieved in a single batch operation cannot exceed 16 MB.
- **Unprocessed Keys:** If any part of your batch request cannot be processed due to capacity unit limitations or internal server errors, those items are returned in the `UnprocessedKeys` response. You need to handle these cases by retrying the failed requests.
- **Capacity Unit Consumption:** Each item read in the batch consumes read capacity units (RCUs). When specifying `ConsistentRead` as `True`, the operation consumes twice the standard RCU for each item.

These limitations require careful planning when implementing batch operations to ensure efficient usage of DynamoDB's resources and handling potential partial failures in batch requests.

#### Handling Reserved Words in DynamoDB Expressions

When working with DynamoDB, certain attribute names may coincide with reserved words used by the service. Using these reserved words directly in expressions for operations like GetItem can cause errors. To circumvent this issue, you can use aliases defined in an `ExpressionAttributeNames` map along with a `ProjectionExpression` to specify the attributes you want to retrieve. This approach allows you to safely use reserved words in your DynamoDB operations.

Here's how you can handle reserved words in DynamoDB expressions by combining `ExpressionAttributeNames` with a `ProjectionExpression`:

```python
result = table.get_item(
    Key={'year': 2021, 'title': 'Example Movie'},
    ProjectionExpression='#yr, title',
    ExpressionAttributeNames={'#yr': 'year'}
)
if 'Item' in result:
    print("Movie found with reserved word attribute:", result['Item'])
else:
    print("Movie not found.")
```

Some common reserved words that often require aliasing in DynamoDB include attributes like *Year*, *Date*, *Total*, *Order*, and *Status*. By aliasing these words, you can use them freely in your queries without encountering syntax errors. This practice ensures your database queries are robust and error-free.

### Summary and Looking Ahead

Great job on completing today's lesson on DynamoDB data retrieval techniques! We've delved into the GetItem and BatchGetItem methods, showing you how to fetch specific items and multiple items efficiently. These techniques are vital for effective data management in DynamoDB, accommodating various data access patterns within your applications. Now, take this knowledge into practice with the exercises provided, applying these methods to diverse scenarios. In our upcoming sessions, we'll explore further into DynamoDB's capabilities, focusing next on its powerful update and delete functionalities to enhance your data management skills. Stay engaged and happy coding!

## Running Scripts and Manipulating DynamoDB Tables

In this task, you are required to run an existing Python script that will create a table named Movies in DynamoDB and populate it with several movie records. Furthermore, the script will retrieve and print specific items from the Movies table. Your primary task is to evaluate the output generated by running the script and understand the workflow of creating a table, adding items to it, and retrieving these items using various methods. No additional modifications or coding are required from you; all you need to do is run the script and comprehend how it operates.

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. However, keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3
from botocore.exceptions import NoCredentialsError

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='Movies',
    KeySchema=[
        { 'AttributeName': 'year', 'KeyType': 'HASH' }, # Partition key
        { 'AttributeName': 'title', 'KeyType': 'RANGE' } # Sort key
    ],
    AttributeDefinitions=[
        { 'AttributeName': 'year', 'AttributeType': 'N' },
        { 'AttributeName': 'title', 'AttributeType': 'S' },
    ],
    ProvisionedThroughput={ 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
)

# Wait for table to be created or until 20 seconds have passed
table.meta.client.get_waiter('table_exists').wait(
    TableName='Movies',
    WaiterConfig={
        'Delay': 2,  # Poll every 2 seconds
        'MaxAttempts': 10  # Stop after 20 seconds, regardless of the result
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})
table.put_item(Item={'year': 2017, 'title': 'Yet Another Movie'})
table.put_item(Item={'year': 2017, 'title': 'One More Movie'})
table.put_item(Item={'year': 2015, 'title': 'An Old Movie'})
table.put_item(Item={'year': 2018, 'title': 'Another New Movie'})

# Get an item using GetItem
result = table.get_item(Key={'year': 2016, 'title': 'The Big New Movie'})
print(result.get('Item'))

# Using BatchGetItem 
response = dynamodb.batch_get_item(
    RequestItems={
        'Movies': {
            'Keys': [
                { 'year': 2016, 'title': 'The Big New Movie' },
                { 'year': 2017, 'title': 'The Bigger, Newer Movie' }
            ],
            'ConsistentRead': True

        }
    }
)
print(response['Responses']['Movies'])

```

### Script Output Evaluation

Running the provided Python script performs the following sequence of operations on DynamoDB:

1. **Creating the `Movies` Table:**  
   The script initiates the creation of a DynamoDB table named `Movies` with `year` as the partition key and `title` as the sort key. It specifies the necessary attribute definitions and provisioned throughput settings.

2. **Waiting for Table Creation:**  
   The script waits until the `Movies` table is successfully created or until 20 seconds have elapsed. This ensures that subsequent operations can be performed on the newly created table.

3. **Inserting Movie Records:**  
   Six movie records are added to the `Movies` table using the `put_item` method. Each record includes a `year` and a `title`.

4. **Retrieving a Single Item with `GetItem`:**  
   The script retrieves the movie titled *"The Big New Movie"* released in 2016 using the `get_item` method and prints the result.

5. **Retrieving Multiple Items with `BatchGetItem`:**  
   It fetches two movies—*“The Big New Movie”* (2016) and *“The Bigger, Newer Movie”* (2017)—using the `batch_get_item` method and prints the retrieved items.

#### Expected Output

Upon executing the script, the following output is generated:

```python
{'year': 2016, 'title': 'The Big New Movie'}
```

```python
[
    {'year': 2016, 'title': 'The Big New Movie'},
    {'year': 2017, 'title': 'The Bigger, Newer Movie'}
]
```

#### Breakdown of the Output

1. **Single Item Retrieval (`GetItem`):**
   
   ```python
   result = table.get_item(Key={'year': 2016, 'title': 'The Big New Movie'})
   print(result.get('Item'))
   ```
   
   **Output:**
   
   ```python
   {'year': 2016, 'title': 'The Big New Movie'}
   ```
   
   This output confirms that the script successfully retrieved the specified movie from the `Movies` table using its primary key.

2. **Multiple Items Retrieval (`BatchGetItem`):**
   
   ```python
   response = dynamodb.batch_get_item(
       RequestItems={
           'Movies': {
               'Keys': [
                   { 'year': 2016, 'title': 'The Big New Movie' },
                   { 'year': 2017, 'title': 'The Bigger, Newer Movie' }
               ],
               'ConsistentRead': True
           }
       }
   )
   print(response['Responses']['Movies'])
   ```
   
   **Output:**
   
   ```python
   [
       {'year': 2016, 'title': 'The Big New Movie'},
       {'year': 2017, 'title': 'The Bigger, Newer Movie'}
   ]
   ```
   
   This output indicates that the script successfully retrieved both specified movies in a single batch operation. Using `BatchGetItem` enhances efficiency by allowing multiple items to be fetched simultaneously.

### Understanding the Workflow

The script demonstrates a typical workflow for interacting with DynamoDB using Python's Boto3 library:

1. **Table Creation:**  
   Establishes the structure of the table by defining primary keys and setting throughput capacities.

2. **Data Insertion:**  
   Populates the table with records, each uniquely identified by the combination of `year` and `title`.

3. **Data Retrieval:**  
   Utilizes both `GetItem` for single-item access and `BatchGetItem` for retrieving multiple items, showcasing flexible data access patterns.

By following this workflow, you can effectively manage and interact with data in DynamoDB, ensuring efficient data storage and retrieval tailored to your application's needs.

## Ensuring Strong Consistency in DynamoDB Read Operations

Great job making it this way! Now you'll modify an existing Python script that creates a DynamoDB table named Movies, inserts items into it, and retrieves those items. The task is to adjust the read consistency of the get_item and batch_get_item methods to use strong consistency. This ensures that the most recent data is always returned from the table. After making these changes, execute the script and observe the implications of using strong consistency for data retrieval.

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, use the reset button located in the top right corner. Remember, resetting will erase any code changes you've made. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'year', 'KeyType': 'HASH'},
            {'AttributeName': 'title', 'KeyType': 'RANGE'}
        ],
        AttributeDefinitions=[
            {'AttributeName': 'year', 'AttributeType': 'N'},
            {'AttributeName': 'title', 'AttributeType': 'S'},
        ],
        ProvisionedThroughput={'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
    )

# Wait for table to be created with a maximum of 10 attempts every 2 seconds
table.meta.client.get_waiter('table_exists').wait(
    TableName='Movies',
    WaiterConfig={
        'Delay': 2,
        'MaxAttempts': 10
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})

# Get an item using GetItem
# TODO: Add strong consistency preference here
result = table.get_item(Key={'year': 2016, 'title': 'The Big New Movie'})
print(result.get('Item'))

# Using BatchGetItem
# TODO: Add strong consistency preference here
response = dynamodb.batch_get_item(
    RequestItems={
        'Movies': {
            'Keys': [
                {'year': 2016, 'title': 'The Big New Movie'},
                {'year': 2017, 'title': 'The Bigger, Newer Movie'}
            ],
        }
    }
)
print(response['Responses']['Movies'])


```

### Ensuring Strong Consistency in DynamoDB Read Operations

Great job making it this far! In this task, we'll modify the existing Python script to ensure that all read operations use strong consistency. This adjustment guarantees that the most recent data is always returned from the `Movies` table.

#### Modified Python Script

Below is the updated script with the necessary changes to enforce strong consistency in both the `get_item` and `batch_get_item` methods:

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'year', 'KeyType': 'HASH'},
            {'AttributeName': 'title', 'KeyType': 'RANGE'}
        ],
        AttributeDefinitions=[
            {'AttributeName': 'year', 'AttributeType': 'N'},
            {'AttributeName': 'title', 'AttributeType': 'S'},
        ],
        ProvisionedThroughput={'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
    )

# Wait for table to be created with a maximum of 10 attempts every 2 seconds
table.meta.client.get_waiter('table_exists').wait(
    TableName='Movies',
    WaiterConfig={
        'Delay': 2,
        'MaxAttempts': 10
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})

# Get an item using GetItem with strong consistency
result = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'},
    ConsistentRead=True  # Enforcing strong consistency
)
print(result.get('Item'))

# Using BatchGetItem with strong consistency
response = dynamodb.batch_get_item(
    RequestItems={
        'Movies': {
            'Keys': [
                {'year': 2016, 'title': 'The Big New Movie'},
                {'year': 2017, 'title': 'The Bigger, Newer Movie'}
            ],
            'ConsistentRead': True  # Enforcing strong consistency
        }
    }
)
print(response['Responses']['Movies'])
```

#### Changes Implemented

1. **Strongly Consistent `GetItem`:**

    ```python
    result = table.get_item(
        Key={'year': 2016, 'title': 'The Big New Movie'},
        ConsistentRead=True  # Enforcing strong consistency
    )
    ```

    - Added the `ConsistentRead=True` parameter to ensure that the read operation returns the most recent data.

2. **Strongly Consistent `BatchGetItem`:**

    ```python
    response = dynamodb.batch_get_item(
        RequestItems={
            'Movies': {
                'Keys': [
                    {'year': 2016, 'title': 'The Big New Movie'},
                    {'year': 2017, 'title': 'The Bigger, Newer Movie'}
                ],
                'ConsistentRead': True  # Enforcing strong consistency
            }
        }
    )
    ```

    - Included the `ConsistentRead=True` parameter within the `RequestItems` to ensure that all items retrieved are the latest.

#### Executing the Modified Script

Upon running the modified script, you can expect the following output:

```python
{'year': 2016, 'title': 'The Big New Movie'}
```

```python
[
    {'year': 2016, 'title': 'The Big New Movie'},
    {'year': 2017, 'title': 'The Bigger, Newer Movie'}
]
```

#### Implications of Using Strong Consistency

1. **Data Accuracy:**
   
   - **Benefit:** Ensures that read operations return the most up-to-date data, reflecting all prior write operations.
   - **Use Case:** Critical for applications where it's essential to retrieve the latest information, such as financial transactions or real-time analytics.

2. **Increased Read Capacity Consumption:**
   
   - **Impact:** Strongly consistent reads consume more read capacity units (RCUs) compared to eventually consistent reads. Specifically, each strongly consistent read requires twice the RCUs of an eventually consistent read.
   - **Consideration:** Plan your provisioned throughput accordingly to accommodate the increased usage, especially in high-traffic applications.

3. **Potential Latency:**
   
   - **Impact:** May experience slightly higher latency as the system ensures data consistency across all replicas before returning the result.
   - **Consideration:** While generally minimal, it's important to assess whether the trade-off between consistency and latency aligns with your application's requirements.

4. **Cost Implications:**
   
   - **Impact:** Due to higher RCU consumption, using strong consistency can lead to increased costs, especially for applications with heavy read operations.
   - **Consideration:** Evaluate the necessity of strong consistency for your use case to balance performance and cost effectively.

#### Summary

By enforcing strong consistency in your read operations, you ensure that your application always accesses the most recent data from DynamoDB. While this enhances data accuracy and reliability, it's essential to be mindful of the associated costs and resource consumption. Assess your application's specific needs to determine the optimal consistency model, leveraging strong consistency where it's most beneficial.

## Mastering Single Item Retrieval

Great progress! In this task, you'll enhance a Python script that already creates a DynamoDB table named Movies and populates it with several movie records. Your objective is to refine this script by implementing read operations that retrieve specific movie details using their primary keys. You will modify the script to retrieve details for two movies: 'The Big New Movie' from 2016, and 'The Bigger, Newer Movie' from 2017. For the second movie, ensure you only retrieve the title by using a projection. After modifying the script, run it to observe how DynamoDB fetches and displays these records.

Important Note: Running scripts can modify resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. Keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            { 'AttributeName': 'year', 'KeyType': 'HASH' }, # Partition key
            { 'AttributeName': 'title', 'KeyType': 'RANGE' }  # Sort key
        ],
        AttributeDefinitions=[
            { 'AttributeName': 'year', 'AttributeType': 'N' },
            { 'AttributeName': 'title', 'AttributeType': 'S' },
        ],
        ProvisionedThroughput={ 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
    )

# Wait for the table to be created or until 20 seconds have passed
dynamodb.meta.client.get_waiter('table_exists').wait(
    TableName='Movies',
    WaiterConfig={
        'Delay': 2,  # Poll every 2 seconds
        'MaxAttempts': 10  # Stop after 20 seconds
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})

# TODO: Add the GetItem operation to retrieve the first movie using its primary key

# TODO: Add the GetItem operation with ProjectionExpression to retrieve only the "title" attribute of the second movie

```

### Enhancing the Python Script with Specific GetItem Operations

Great progress! We'll now refine the existing Python script to include two `GetItem` operations:

1. **Retrieve the first movie** (`'The Big New Movie'` from `2016`) using its primary key.
2. **Retrieve only the title** of the second movie (`'The Bigger, Newer Movie'` from `2017`) using a `ProjectionExpression`.

#### Modified Python Script

Below is the updated script with the required `GetItem` operations implemented:

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            { 'AttributeName': 'year', 'KeyType': 'HASH' },  # Partition key
            { 'AttributeName': 'title', 'KeyType': 'RANGE' }  # Sort key
        ],
        AttributeDefinitions=[
            { 'AttributeName': 'year', 'AttributeType': 'N' },
            { 'AttributeName': 'title', 'AttributeType': 'S' },
        ],
        ProvisionedThroughput={ 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
    )

# Wait for the table to be created or until 20 seconds have passed
dynamodb.meta.client.get_waiter('table_exists').wait(
    TableName='Movies',
    WaiterConfig={
        'Delay': 2,  # Poll every 2 seconds
        'MaxAttempts': 10  # Stop after 20 seconds
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})

# Get an item using GetItem to retrieve the first movie using its primary key
result_first_movie = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'},
    ConsistentRead=True  # Ensuring strong consistency
)
print("First Movie Details:", result_first_movie.get('Item'))

# Get an item using GetItem with ProjectionExpression to retrieve only the "title" of the second movie
result_second_movie = table.get_item(
    Key={'year': 2017, 'title': 'The Bigger, Newer Movie'},
    ProjectionExpression='title',  # Retrieving only the title attribute
    ConsistentRead=True  # Ensuring strong consistency
)
print("Second Movie Title:", result_second_movie.get('Item'))
```

#### Changes Implemented

1. **Retrieving the First Movie (`GetItem`):**
   
   ```python
   result_first_movie = table.get_item(
       Key={'year': 2016, 'title': 'The Big New Movie'},
       ConsistentRead=True  # Ensuring strong consistency
   )
   print("First Movie Details:", result_first_movie.get('Item'))
   ```
   
   - **Purpose:** Retrieves the complete details of `'The Big New Movie'` released in `2016`.
   - **Parameters:**
     - `Key`: Specifies the primary key (`year` and `title`) to identify the item.
     - `ConsistentRead=True`: Ensures that the read operation returns the most recent data.

2. **Retrieving Only the Title of the Second Movie (`GetItem` with `ProjectionExpression`):**
   
   ```python
   result_second_movie = table.get_item(
       Key={'year': 2017, 'title': 'The Bigger, Newer Movie'},
       ProjectionExpression='title',  # Retrieving only the title attribute
       ConsistentRead=True  # Ensuring strong consistency
   )
   print("Second Movie Title:", result_second_movie.get('Item'))
   ```
   
   - **Purpose:** Retrieves only the `title` attribute of `'The Bigger, Newer Movie'` released in `2017`.
   - **Parameters:**
     - `Key`: Specifies the primary key (`year` and `title`) to identify the item.
     - `ProjectionExpression='title'`: Limits the response to include only the `title` attribute.
     - `ConsistentRead=True`: Ensures that the read operation returns the most recent data.

#### Expected Output

Upon executing the modified script, you should observe the following output:

```python
First Movie Details: {'year': 2016, 'title': 'The Big New Movie'}
```

```python
Second Movie Title: {'title': 'The Bigger, Newer Movie'}
```

#### Understanding the Enhancements

1. **Strong Consistency (`ConsistentRead=True`):**
   
   - **Benefit:** Guarantees that the data read is the most up-to-date, reflecting all prior write operations.
   - **Use Case:** Essential for applications requiring the latest data, such as financial transactions or real-time analytics.
   
2. **ProjectionExpression:**
   
   - **Purpose:** Allows you to retrieve only specific attributes of an item, reducing the amount of data transferred and potentially lowering costs.
   - **Usage in Script:** By specifying `ProjectionExpression='title'`, the second `GetItem` operation fetches only the `title` of the movie, omitting other attributes.

3. **Output Clarity:**
   
   - **First Movie Retrieval:** Provides a complete view of the movie details.
   - **Second Movie Retrieval:** Focuses solely on the `title`, demonstrating how to access specific attributes efficiently.

#### Implications of the Enhancements

- **Performance and Cost Efficiency:**
  
  Using `ProjectionExpression` can lead to performance improvements and cost savings by limiting the data retrieved to only what is necessary.

- **Data Accuracy:**
  
  Ensuring strong consistency is crucial for scenarios where data accuracy is paramount. However, it is important to balance this with the increased read capacity consumption associated with strongly consistent reads.

- **Scalability:**
  
  These enhancements contribute to building scalable applications by optimizing data retrieval operations based on specific needs.

#### Summary

By implementing targeted `GetItem` operations with strong consistency and attribute projection, the script now efficiently retrieves specific movie details from the DynamoDB `Movies` table. These modifications not only enhance data retrieval specificity but also illustrate best practices for optimizing DynamoDB interactions based on application requirements.Executed 1st Code Block

## Retrieving Multiple Items in DynamoDB

Good job! This task will test your understanding of DynamoDB data retrieval operations. You have a script that builds a table named Movies and populates it with a few records. Each record represents a movie, containing attributes such as year and title. Your objective is to extend the functionality of this script by adding data retrieval operations to fetch movies from the table. Specifically, you must add the batch_get_item operation to retrieve two movies simultaneously: 'The Big New Movie' from 2016 and 'The Bigger, Newer Movie' from 2017.

Important Note: Running scripts can modify the resources in our AWS simulator. To revert to the initial state, you can use the reset button located in the top right corner. However, keep in mind that resetting will erase any code changes. To preserve your code during a reset, consider copying it to the clipboard.

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Table creation
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            { 'AttributeName': 'year', 'KeyType': 'HASH' }, # Partition key 
            { 'AttributeName': 'title', 'KeyType': 'RANGE' }  # Sort key
        ],
        AttributeDefinitions=[
            { 'AttributeName': 'year', 'AttributeType': 'N' },
            { 'AttributeName': 'title', 'AttributeType': 'S' },
        ],
        ProvisionedThroughput={ 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
    )

# Wait for table to be created, polling every 2 seconds and making 10 attempts maximum
table.meta.client.get_waiter('table_exists').wait(TableName='Movies', WaiterConfig={'Delay': 2, 'MaxAttempts': 10})

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})
table.put_item(Item={'year': 2017, 'title': 'Yet Another Movie'})
table.put_item(Item={'year': 2017, 'title': 'One More Movie'})
table.put_item(Item={'year': 2015, 'title': 'An Old Movie'})
table.put_item(Item={'year': 2018, 'title': 'Another New Movie'})

# TODO: Add BatchGetItem operation to fetch 'The Big New Movie' and 'The Bigger, Newer Movie'
# Print retrieved items

```

### Enhancing the Python Script with BatchGetItem Operation

Great progress! We'll now extend the existing Python script to include a `BatchGetItem` operation that retrieves two specific movies simultaneously: *"The Big New Movie"* from `2016` and *"The Bigger, Newer Movie"* from `2017`. This enhancement demonstrates how to efficiently fetch multiple items from a DynamoDB table in a single request.

#### Modified Python Script

Below is the updated script with the added `BatchGetItem` operation:

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')  # Specify your AWS region

# Table creation
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {'AttributeName': 'year', 'KeyType': 'HASH'},   # Partition key 
            {'AttributeName': 'title', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'year', 'AttributeType': 'N'},
            {'AttributeName': 'title', 'AttributeType': 'S'},
        ],
        ProvisionedThroughput={'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5}
    )

# Wait for the table to be created, polling every 2 seconds and making 10 attempts maximum
table.meta.client.get_waiter('table_exists').wait(
    TableName='Movies',
    WaiterConfig={'Delay': 2, 'MaxAttempts': 10}
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})
table.put_item(Item={'year': 2017, 'title': 'Yet Another Movie'})
table.put_item(Item={'year': 2017, 'title': 'One More Movie'})
table.put_item(Item={'year': 2015, 'title': 'An Old Movie'})
table.put_item(Item={'year': 2018, 'title': 'Another New Movie'})

# Add BatchGetItem operation to fetch 'The Big New Movie' and 'The Bigger, Newer Movie'
response = dynamodb.batch_get_item(
    RequestItems={
        'Movies': {
            'Keys': [
                {'year': 2016, 'title': 'The Big New Movie'},
                {'year': 2017, 'title': 'The Bigger, Newer Movie'}
            ],
            'ConsistentRead': True  # Ensuring strong consistency
        }
    }
)

# Print retrieved items
retrieved_movies = response['Responses']['Movies']
print("Retrieved Movies:")
for movie in retrieved_movies:
    print(movie)
```

#### Changes Implemented

1. **Specify AWS Region:**

    ```python
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')  # Specify your AWS region
    ```
    
    - **Purpose:** Adding the `region_name` parameter ensures that the DynamoDB resource connects to the correct AWS region, preventing potential `NoRegionError` exceptions.
    - **Note:** Replace `'us-west-2'` with the appropriate region for your setup.

2. **Adding `BatchGetItem` Operation:**

    ```python
    response = dynamodb.batch_get_item(
        RequestItems={
            'Movies': {
                'Keys': [
                    {'year': 2016, 'title': 'The Big New Movie'},
                    {'year': 2017, 'title': 'The Bigger, Newer Movie'}
                ],
                'ConsistentRead': True  # Ensuring strong consistency
            }
        }
    )
    
    # Print retrieved items
    retrieved_movies = response['Responses']['Movies']
    print("Retrieved Movies:")
    for movie in retrieved_movies:
        print(movie)
    ```
    
    - **Purpose:** The `batch_get_item` method retrieves multiple items from the `Movies` table in a single request. This approach is more efficient than making separate `GetItem` calls for each movie.
    - **Parameters:**
        - `RequestItems`: Specifies the table name and the list of primary keys (`year` and `title`) identifying the items to retrieve.
        - `ConsistentRead=True`: Ensures that the read operations return the most up-to-date data.
    - **Output Handling:** The retrieved items are accessed via `response['Responses']['Movies']` and printed in a readable format.

#### Expected Output

Upon executing the modified script, you should observe an output similar to the following:

```python
Retrieved Movies:
{'year': 2016, 'title': 'The Big New Movie'}
{'year': 2017, 'title': 'The Bigger, Newer Movie'}
```

#### Understanding the Enhancements

1. **BatchGetItem Operation:**
   
    - **Efficiency:** Fetching multiple items in a single `BatchGetItem` request reduces the number of network calls, resulting in faster data retrieval and lower latency compared to individual `GetItem` requests.
    - **Strong Consistency:** By setting `ConsistentRead=True`, the operation ensures that the retrieved items reflect all prior write operations, providing accurate and reliable data.

2. **Error Handling Considerations:**
   
    - **Unprocessed Keys:** While not handled in this script, it's important to note that `batch_get_item` may return unprocessed keys if the request exceeds DynamoDB's capacity limits. In a production environment, you should implement logic to retry fetching these unprocessed keys.
    - **Provisioned Throughput:** Ensure that your DynamoDB table's provisioned throughput is adequately configured to handle the read and write operations, especially when using strong consistency modes.

3. **AWS Region Specification:**
   
    - **Importance:** Specifying the `region_name` is crucial to direct the resource to the correct AWS region, avoiding connectivity issues and ensuring compliance with data residency requirements.
    - **Customization:** Adjust the region according to where your DynamoDB table is hosted or your organization's best practices.

#### Summary

By incorporating the `BatchGetItem` operation into your Python script, you enhance the efficiency of data retrieval from the DynamoDB `Movies` table. This approach allows for simultaneous fetching of multiple items, ensuring that your application can access necessary data swiftly and reliably. Additionally, specifying the AWS region prevents common configuration errors, ensuring seamless interaction with your DynamoDB resources.

Deploying these best practices in your DynamoDB interactions will contribute to building scalable, efficient, and robust applications tailored to your data management needs.

## Mastering DynamoDB Operations in Python

Great job so far! In this task, you will continue to build upon your foundational knowledge of DynamoDB by writing Python scripts that not only demonstrate your ability to interact with DynamoDB tables but also to manipulate and retrieve data efficiently. You will begin with a template that already includes scripts for creating a DynamoDB table named Movies and populating it with several records. Your focus will be on expanding this script to include data retrieval operations using GetItem and BatchGetItem. Specifically, you will demonstrate different retrieval strategies including simple reads, reads with projection, and ensuring strongly consistent reads.

Important Note: Keep in mind that running scripts can modify the resources in our AWS simulator. If you need to revert your AWS environment to its initial state, use the reset button located in the top right corner of the simulator interface. However, resetting the environment will remove any code changes you've made during your session. To avoid losing your work, remember to save your code externally before hitting the reset button.

```python
import boto3

# Create DynamoDB resource
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            { 'AttributeName': 'year', 'KeyType': 'HASH' }, # Partition key
            { 'AttributeName': 'title', 'KeyType': 'RANGE' } # Sort key
        ],
        AttributeDefinitions=[
            { 'AttributeName': 'year', 'AttributeType': 'N' },
            { 'AttributeName': 'title', 'AttributeType': 'S' },
        ],
        ProvisionedThroughput={ 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
    )

# Wait for table to be created with custom waiter
table.meta.client.get_waiter('table_exists').wait(
    TableName='Movies', 
    WaiterConfig={
        'Delay': 2,           # Poll every 2 seconds
        'MaxAttempts': 10     # Make maximum 10 attempts
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})
table.put_item(Item={'year': 2017, 'title': 'Yet Another Movie'})
table.put_item(Item={'year': 2017, 'title': 'One More Movie'})
table.put_item(Item={'year': 2015, 'title': 'An Old Movie'})
table.put_item(Item={'year': 2018, 'title': 'Another New Movie'})

# TODO: Retrieve 'The Big New Movie' from 2016 using a simple GetItem.
# TODO: Retrieve 'The Big New Movie' from 2016 using GetItem with ProjectionExpression for 'title' and 'genre'.
# TODO: Retrieve 'The Big New Movie' from 2016 using a strongly consistent read.
# TODO: Use BatchGetItem to retrieve 'The Big New Movie' from 2016 and 'The Bigger, Newer Movie' from 2017 with consistent read.

```

### Mastering DynamoDB Operations in Python

Building upon your foundational knowledge of DynamoDB, we'll enhance the existing Python script to include various data retrieval operations. This will demonstrate different strategies for interacting with DynamoDB tables, such as simple reads, reads with projection, and ensuring strongly consistent reads.

#### Modified Python Script

Below is the updated script with the required `GetItem` and `BatchGetItem` operations implemented:

```python
import boto3

# Create DynamoDB resource with specified AWS region
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')  # Replace 'us-west-2' with your AWS region

# Create the DynamoDB table.
table_name = 'Movies'
table_names = [table.name for table in dynamodb.tables.all()]

if table_name in table_names:
    table = dynamodb.Table(table_name)
else:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            { 'AttributeName': 'year', 'KeyType': 'HASH' },  # Partition key
            { 'AttributeName': 'title', 'KeyType': 'RANGE' }  # Sort key
        ],
        AttributeDefinitions=[
            { 'AttributeName': 'year', 'AttributeType': 'N' },
            { 'AttributeName': 'title', 'AttributeType': 'S' },
        ],
        ProvisionedThroughput={ 'ReadCapacityUnits': 5, 'WriteCapacityUnits': 5 }
    )

# Wait for table to be created with custom waiter
table.meta.client.get_waiter('table_exists').wait(
    TableName='Movies', 
    WaiterConfig={
        'Delay': 2,           # Poll every 2 seconds
        'MaxAttempts': 10     # Make maximum 10 attempts
    }
)

# Insert movies into the table
table.put_item(Item={'year': 2016, 'title': 'The Big New Movie'})
table.put_item(Item={'year': 2017, 'title': 'The Bigger, Newer Movie'})
table.put_item(Item={'year': 2017, 'title': 'Yet Another Movie'})
table.put_item(Item={'year': 2017, 'title': 'One More Movie'})
table.put_item(Item={'year': 2015, 'title': 'An Old Movie'})
table.put_item(Item={'year': 2018, 'title': 'Another New Movie'})

# Retrieve 'The Big New Movie' from 2016 using a simple GetItem.
result_simple = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'}
)
print("Simple GetItem Result:", result_simple.get('Item'))

# Retrieve 'The Big New Movie' from 2016 using GetItem with ProjectionExpression for 'title' and 'genre'.
result_projection = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'},
    ProjectionExpression='title, genre'
)
print("GetItem with ProjectionExpression Result:", result_projection.get('Item'))

# Retrieve 'The Big New Movie' from 2016 using a strongly consistent read.
result_strong = table.get_item(
    Key={'year': 2016, 'title': 'The Big New Movie'},
    ConsistentRead=True
)
print("Strongly Consistent GetItem Result:", result_strong.get('Item'))

# Use BatchGetItem to retrieve 'The Big New Movie' from 2016 and 'The Bigger, Newer Movie' from 2017 with consistent read.
response_batch = dynamodb.batch_get_item(
    RequestItems={
        'Movies': {
            'Keys': [
                {'year': 2016, 'title': 'The Big New Movie'},
                {'year': 2017, 'title': 'The Bigger, Newer Movie'}
            ],
            'ConsistentRead': True
        }
    }
)
retrieved_movies = response_batch['Responses']['Movies']
print("BatchGetItem Result:")
for movie in retrieved_movies:
    print(movie)
```

#### Implemented Retrieval Operations

1. **Simple GetItem**

    ```python
    # Retrieve 'The Big New Movie' from 2016 using a simple GetItem.
    result_simple = table.get_item(
        Key={'year': 2016, 'title': 'The Big New Movie'}
    )
    print("Simple GetItem Result:", result_simple.get('Item'))
    ```

    - **Purpose:** Fetches the complete item for `'The Big New Movie'` released in `2016`.
    - **Parameters:**
        - `Key`: Specifies the primary key (`year` and `title`) to identify the item.
    - **Output:** Prints the entire movie record.

2. **GetItem with ProjectionExpression**

    ```python
    # Retrieve 'The Big New Movie' from 2016 using GetItem with ProjectionExpression for 'title' and 'genre'.
    result_projection = table.get_item(
        Key={'year': 2016, 'title': 'The Big New Movie'},
        ProjectionExpression='title, genre'
    )
    print("GetItem with ProjectionExpression Result:", result_projection.get('Item'))
    ```

    - **Purpose:** Fetches only the `title` and `genre` attributes of `'The Big New Movie'` released in `2016`.
    - **Parameters:**
        - `Key`: Specifies the primary key.
        - `ProjectionExpression`: Limits the retrieved attributes to `title` and `genre`.
    - **Note:** Since the `genre` attribute was not inserted during the `put_item` operations, it will not be present in the result.
    - **Output:** Prints the retrieved attributes.

3. **Strongly Consistent GetItem**

    ```python
    # Retrieve 'The Big New Movie' from 2016 using a strongly consistent read.
    result_strong = table.get_item(
        Key={'year': 2016, 'title': 'The Big New Movie'},
        ConsistentRead=True
    )
    print("Strongly Consistent GetItem Result:", result_strong.get('Item'))
    ```

    - **Purpose:** Ensures that the read operation returns the most up-to-date data for `'The Big New Movie'` released in `2016`.
    - **Parameters:**
        - `Key`: Specifies the primary key.
        - `ConsistentRead=True`: Forces the read to be strongly consistent.
    - **Output:** Prints the movie record with the latest data.

4. **BatchGetItem with Consistent Read**

    ```python
    # Use BatchGetItem to retrieve 'The Big New Movie' from 2016 and 'The Bigger, Newer Movie' from 2017 with consistent read.
    response_batch = dynamodb.batch_get_item(
        RequestItems={
            'Movies': {
                'Keys': [
                    {'year': 2016, 'title': 'The Big New Movie'},
                    {'year': 2017, 'title': 'The Bigger, Newer Movie'}
                ],
                'ConsistentRead': True
            }
        }
    )
    retrieved_movies = response_batch['Responses']['Movies']
    print("BatchGetItem Result:")
    for movie in retrieved_movies:
        print(movie)
    ```

    - **Purpose:** Retrieves both `'The Big New Movie'` from `2016` and `'The Bigger, Newer Movie'` from `2017` in a single batch operation.
    - **Parameters:**
        - `RequestItems`: Specifies the table and the list of primary keys to retrieve.
        - `ConsistentRead=True`: Ensures that the read operations are strongly consistent.
    - **Output:** Prints the retrieved movie records.

#### Expected Output

Upon executing the modified script, you should observe the following output:

```python
Simple GetItem Result: {'year': 2016, 'title': 'The Big New Movie'}
```

```python
GetItem with ProjectionExpression Result: {'title': 'The Big New Movie'}
```

```python
Strongly Consistent GetItem Result: {'year': 2016, 'title': 'The Big New Movie'}
```

```python
BatchGetItem Result:
{'year': 2016, 'title': 'The Big New Movie'}
{'year': 2017, 'title': 'The Bigger, Newer Movie'}
```

#### Breakdown of the Output

1. **Simple GetItem Result**

    ```python
    {'year': 2016, 'title': 'The Big New Movie'}
    ```

    - **Explanation:** Successfully retrieved the entire record for `'The Big New Movie'` released in `2016`.

2. **GetItem with ProjectionExpression Result**

    ```python
    {'title': 'The Big New Movie'}
    ```

    - **Explanation:** Retrieved only the `title` attribute of `'The Big New Movie'`. The `genre` attribute is absent because it was not added during the data insertion phase.

3. **Strongly Consistent GetItem Result**

    ```python
    {'year': 2016, 'title': 'The Big New Movie'}
    ```

    - **Explanation:** Retrieved the most up-to-date data for `'The Big New Movie'` using a strongly consistent read.

4. **BatchGetItem Result**

    ```python
    {'year': 2016, 'title': 'The Big New Movie'}
    {'year': 2017, 'title': 'The Bigger, Newer Movie'}
    ```

    - **Explanation:** Successfully retrieved both specified movies in a single batch operation with strong consistency.

#### Understanding the Enhancements

1. **Simple GetItem**

    - **Functionality:** Allows retrieval of a single item based on its primary key.
    - **Use Case:** Ideal for scenarios where you need to access specific records without additional constraints.

2. **GetItem with ProjectionExpression**

    - **Functionality:** Enables selective retrieval of specific attributes within an item.
    - **Advantages:**
        - **Performance:** Reduces the amount of data transferred, leading to faster response times.
        - **Cost-Efficiency:** Minimizes read capacity unit (RCU) consumption by fetching only necessary attributes.
    - **Considerations:** Ensure that the projected attributes exist; otherwise, they will be omitted from the result.

3. **Strongly Consistent GetItem**

    - **Functionality:** Guarantees that the data read reflects all prior writes, ensuring data accuracy.
    - **Advantages:**
        - **Data Reliability:** Essential for applications where data integrity is critical, such as financial systems.
    - **Trade-offs:**
        - **Higher RCU Consumption:** Strongly consistent reads consume twice the RCUs of eventually consistent reads.
        - **Potentially Higher Latency:** May experience slightly increased latency due to the consistency guarantee.

4. **BatchGetItem with Consistent Read**

    - **Functionality:** Allows retrieval of multiple items in a single request with strong consistency.
    - **Advantages:**
        - **Efficiency:** Reduces the number of network calls, enhancing performance.
        - **Consistency:** Ensures that all retrieved items are up-to-date.
    - **Considerations:**
        - **Handling Unprocessed Keys:** Implement logic to retry fetching any unprocessed keys returned by DynamoDB.
        - **Provisioned Throughput Management:** Ensure adequate read capacity to handle batch requests without throttling.

#### Best Practices and Considerations

- **Attribute Definitions:** Ensure that all attributes you intend to project or retrieve exist in your DynamoDB items to avoid missing data in your results.
  
- **Provisioned Throughput:** Monitor and adjust your table's read and write capacity units based on your application's usage patterns to prevent throttling and ensure optimal performance.
  
- **Error Handling:** Implement robust error handling, especially for batch operations, to gracefully manage scenarios where some items cannot be retrieved in a single request.
  
- **AWS Region Specification:** Always specify the appropriate AWS region when creating DynamoDB resources to avoid connectivity issues and ensure compliance with data residency requirements.

#### Summary

By incorporating these data retrieval operations into your Python script, you've enhanced your ability to interact with DynamoDB tables effectively. Understanding and implementing different retrieval strategies—ranging from simple reads to batch operations with strong consistency—enables you to build scalable and efficient applications tailored to diverse data access needs. Remember to consider the trade-offs associated with each approach, such as consistency guarantees and resource consumption, to make informed decisions aligned with your application's requirements.Executed 1st Code Block